Customer Analysis

In [2]:
import pandas as pd
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_excel('SStore_data.xls')

In [6]:
orders.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [19]:
import datetime as dt
NOW = dt.datetime(2017,12,31)

In [20]:
# Make the date_placed column datetime
df['Order Date']= pd.to_datetime(df['Order Date'])


In [23]:
rfmTable = df.groupby('Customer Name').agg({'Order Date': lambda x: (NOW - x.max()).days, # Recency
                                        'Order ID': lambda x: len(x),      # Frequency
                                        'Sales': lambda x: x.sum()}) # Monetary Value



In [26]:
rfmTable['Order Date'] = rfmTable['Order Date'].astype(int)
rfmTable.rename(columns={'Order Date': 'Recency', 
                         'Order ID': 'Frequency', 
                         'Sales': 'MoneyValue'}, inplace=True)

KeyError: 'Order Date'

In [27]:
rfmTable.head()

,recency,frequency,monetary_value
Customer Name,,,
Aaron Bergman,416,6,886.156
Aaron Hawkins,13,11,1744.700
Aaron Smayling,89,10,3050.692
Adam Bellavance,55,18,7755.620
Adam Hart,35,20,3250.337


In [28]:
aaron = df[df['Customer Name']=='Aaron Bergman']
aaron

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
4961,4962,CA-2014-156587,2014-03-07,2014-03-08,First Class,AB-10015,Aaron Bergman,Consumer,United States,Seattle,...,98103,West,FUR-CH-10004477,Furniture,Chairs,"Global Push Button Manager's Chair, Indigo",48.712,1,0.2,5.4801
4962,4963,CA-2014-156587,2014-03-07,2014-03-08,First Class,AB-10015,Aaron Bergman,Consumer,United States,Seattle,...,98103,West,OFF-AR-10001427,Office Supplies,Art,Newell 330,17.940,3,0.0,4.6644
4963,4964,CA-2014-156587,2014-03-07,2014-03-08,First Class,AB-10015,Aaron Bergman,Consumer,United States,Seattle,...,98103,West,OFF-ST-10002344,Office Supplies,Storage,"Carina 42""Hx23 3/4""W Media Storage Unit",242.940,3,0.0,4.8588
8222,8223,CA-2014-152905,2014-02-18,2014-02-24,Standard Class,AB-10015,Aaron Bergman,Consumer,United States,Arlington,...,76017,Central,OFF-ST-10000321,Office Supplies,Storage,Akro Stacking Bins,12.624,2,0.2,-2.5248
8801,8802,CA-2016-140935,2016-11-10,2016-11-12,First Class,AB-10015,Aaron Bergman,Consumer,United States,Oklahoma City,...,73120,Central,TEC-PH-10000562,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544
8802,8803,CA-2016-140935,2016-11-10,2016-11-12,First Class,AB-10015,Aaron Bergman,Consumer,United States,Oklahoma City,...,73120,Central,FUR-BO-10003966,Furniture,Bookcases,"Sauder Facets Collection Library, Sky Alder Fi...",341.960,2,0.0,54.7136
